In [45]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
from datetime import datetime
from os import environ
import pickle
import sys

import backtrader as bt
import pandas as pd
import pdmongo as pdm

from polaristools.polarisbot import PolarisBot
from strategies import mystrategies

## Instantiate Cerebro

In [46]:
cerebro = bt.Cerebro()

## Configure Cerebro

In [47]:
cerebro.broker.set_cash(100)
cerebro.addsizer(bt.sizers.PercentSizer, percents=25)
cerebro.broker.setcommission(commission=0.005)

## Add data to Cerebro

In [48]:
polaris = PolarisBot()

## <center>Daily Data

In [49]:
# df = polaris.dataframeFromBinary('df_klines_BTCUSDT_1d')
# df = df['2018-08-01':'2022-08-01'].copy()

In [50]:
# feed = bt.feeds.PandasData(dataname=df)
# cerebro.adddata(feed)

## <center>Minute Data

In [51]:
minutes = 60

df = polaris.dataframeFromBinary(f'df_klines_BTCUSDT_{minutes}m')
df = df['2022-05-01':'2022-08-10'].copy()

feed = bt.feeds.PandasData(dataname=df, timeframe=bt.TimeFrame.Minutes, compression=minutes)
cerebro.adddata(feed)

## Add analyzers to Cerebro

In [52]:
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

## Strategy and Parameters range

In [53]:
''' cerebro.optstrategy(mystrategies.EmaCrossTriple, 
    
    enter_long=False,
    enter_short=True,
    
    ema_slow=[100,120],
    ema_mid=[60,70,80],
    ema_fast=[25,35,40],
    
    tp_k=range(1,3),
    sl_k=range(1,3),
    
    # leverage_factor = 1.0
) '''

' cerebro.optstrategy(mystrategies.EmaCrossTriple, \n    \n    enter_long=False,\n    enter_short=True,\n    \n    ema_slow=[100,120],\n    ema_mid=[60,70,80],\n    ema_fast=[25,35,40],\n    \n    tp_k=range(1,3),\n    sl_k=range(1,3),\n    \n    # leverage_factor = 1.0\n) '

In [54]:
cerebro.optstrategy(mystrategies.AroonPlusMa, 
    
    enter_long=True,
    enter_short=True,
    
    ema=range(100,500,25),
    
    aroon_timeperiod=range(50,301,25),
    
    # leverage_factor = 1.0
)

## Run Cerebro

In [55]:
optimizations = cerebro.run()

In [56]:
optimizations.__len__()

176

# Analyze Results

In [57]:
# optimizations[0][0].analyzers.getnames()

In [58]:
all_params = [
    dict(zip(
        list(x[0].params._getkeys()),
        list(x[0].params._getvalues()),
    ))for x in optimizations 
]

In [59]:
len(all_params)

176

In [60]:
df_params = pd.DataFrame(all_params)
df_params.drop(columns=['verbose'], inplace=True)
# df_params.head()

In [61]:
trades = [
    dict(
        pnl_net       = x[0].analyzers.tradeanalyzer.get_analysis()['pnl']['net']['total'], 
        total_trades  = x[0].analyzers.tradeanalyzer.get_analysis()['total']['total'], 
        won           = x[0].analyzers.tradeanalyzer.get_analysis()['won']['total'], 
        lost          = x[0].analyzers.tradeanalyzer.get_analysis()['lost']['total'],
        
        long_won      = x[0].analyzers.tradeanalyzer.get_analysis()['long']['won'],
        short_won     = x[0].analyzers.tradeanalyzer.get_analysis()['short']['won'],
        
        long_lost     = x[0].analyzers.tradeanalyzer.get_analysis()['long']['lost'],
        short_lost    = x[0].analyzers.tradeanalyzer.get_analysis()['short']['lost'],
        
        longs_pnl    = x[0].analyzers.tradeanalyzer.get_analysis()['long']['pnl']['total'],
        shorts_pnl    = x[0].analyzers.tradeanalyzer.get_analysis()['short']['pnl']['total'],
        
        moneydown_max = x[0].analyzers.drawdown.get_analysis()['max']['moneydown'],
    )for x in optimizations
]

In [62]:
df_trades = pd.DataFrame(trades)

In [63]:
df_results = pd.concat([df_params, df_trades], axis=1)

In [64]:
df_results.columns

Index(['futures_like', 'enter_long', 'enter_short', 'ema', 'aroon_timeperiod',
       'leverage_factor', 'margin', 'tp_k', 'sl_k', 'trail', 'pnl_net',
       'total_trades', 'won', 'lost', 'long_won', 'short_won', 'long_lost',
       'short_lost', 'longs_pnl', 'shorts_pnl', 'moneydown_max'],
      dtype='object')

# BEST RESULTS

In [65]:
best = df_results[ 
    [
    'pnl_net',
    'won',
    'lost',
    'total_trades',
    'moneydown_max',
    'long_won',
    'long_lost',
    'short_won',
    'short_lost',
    'longs_pnl',
    'shorts_pnl',
    'leverage_factor',
    'aroon_timeperiod',
    'ema',
    ]
    ].nlargest(10,['pnl_net'])
    
best

,pnl_net,won,lost,total_trades,moneydown_max,long_won,long_lost,short_won,short_lost,longs_pnl,shorts_pnl,leverage_factor,aroon_timeperiod,ema
156,5.710581,3,6,9,4.301086,0,1,3,5,-0.229392,5.939973,1.0,100,450
167,5.710581,3,6,9,4.301086,0,1,3,5,-0.229392,5.939973,1.0,100,475
69,5.320188,3,5,8,5.715897,1,1,2,4,1.640227,3.679961,1.0,125,250
134,5.314077,3,7,10,4.284953,0,1,3,6,-0.228531,5.542608,1.0,100,400
145,5.314077,3,7,10,4.284953,0,1,3,6,-0.228531,5.542608,1.0,100,425
70,3.966604,2,5,8,5.364620,1,1,1,4,-0.890045,4.856649,1.0,150,250
81,3.966604,2,5,8,5.364620,1,1,1,4,-0.890045,4.856649,1.0,150,275
92,3.966604,2,5,8,5.364620,1,1,1,4,-0.890045,4.856649,1.0,150,300
0,3.112332,5,15,21,8.387482,1,3,4,12,0.124332,2.988000,1.0,50,100
37,3.092883,2,4,7,6.225688,1,2,1,2,-2.651095,5.743977,1.0,150,175


In [66]:
dict(df_results.loc[ int(best.first_valid_index()) ])

{'futures_like': True,
 'enter_long': True,
 'enter_short': True,
 'ema': 450,
 'aroon_timeperiod': 100,
 'leverage_factor': 1.0,
 'margin': 0.6,
 'tp_k': 2,
 'sl_k': 2,
 'trail': None,
 'pnl_net': 5.710581466328954,
 'total_trades': 9,
 'won': 3,
 'lost': 6,
 'long_won': 0,
 'short_won': 3,
 'long_lost': 1,
 'short_lost': 5,
 'longs_pnl': -0.22939170887921237,
 'shorts_pnl': 5.939973175208166,
 'moneydown_max': 4.301085505313495}

In [67]:
df_results[
    [
    'pnl_net',
    'won',
    'lost',
    'total_trades',
    'moneydown_max',
    'long_won',
    'long_lost',
    'short_won',
    'short_lost',
    'longs_pnl',
    'shorts_pnl',
    'leverage_factor',
    'aroon_timeperiod',
    'ema',
    ]
].nsmallest(10,'pnl_net')

,pnl_net,won,lost,total_trades,moneydown_max,long_won,long_lost,short_won,short_lost,longs_pnl,shorts_pnl,leverage_factor,aroon_timeperiod,ema
8,-14.345967,0,4,4,15.789499,0,4,0,0,-14.345967,0.000000,1.0,250,100
19,-14.345967,0,4,4,15.789499,0,4,0,0,-14.345967,0.000000,1.0,250,125
30,-14.345967,0,4,4,15.789499,0,4,0,0,-14.345967,0.000000,1.0,250,150
41,-14.345967,0,4,4,15.789499,0,4,0,0,-14.345967,0.000000,1.0,250,175
52,-14.345967,0,4,4,15.789499,0,4,0,0,-14.345967,0.000000,1.0,250,200
63,-14.345967,0,4,4,15.789499,0,4,0,0,-14.345967,0.000000,1.0,250,225
74,-14.345967,0,4,4,15.789499,0,4,0,0,-14.345967,0.000000,1.0,250,250
62,-12.112524,0,4,5,13.875204,0,0,0,4,0.000000,-12.112524,1.0,225,225
73,-12.112524,0,4,5,13.875204,0,0,0,4,0.000000,-12.112524,1.0,225,250
84,-12.112524,0,4,5,13.875204,0,0,0,4,0.000000,-12.112524,1.0,225,275


### <center>Failed test objects

In [68]:
def identify_failed_objects(optimizations):
    failed_idx = []
    for idx,op in enumerate(optimizations):
        try:
            op[0].analyzers.tradeanalyzer.get_analysis()['pnl']['net']['total']
        except:
            failed_idx.append(idx)
    print('Failed tests #',len(failed_idx))
    return failed_idx

def pop_failed_objects(optimizations,failed_idx):
    failed_objects = []
    for f in failed_idx:
        failed_objects.append(optimizations.pop(f))
    print('#',len(failed))
    return failed_objects

In [69]:
# len(optimizations)

In [70]:
''' while True:
    indices = identify_failed_objects(optimizations)
    if not indices:
        break
    objects = pop_failed_objects(optimizations,failed_idx=indices)
    if not objects:
        break

print(len(objects)) '''

' while True:\n    indices = identify_failed_objects(optimizations)\n    if not indices:\n        break\n    objects = pop_failed_objects(optimizations,failed_idx=indices)\n    if not objects:\n        break\n\nprint(len(objects)) '

In [71]:
''' dict(zip(
        list(objects[0][0].params._getkeys()),
        list(objects[0][0].params._getvalues()),
    )) '''

' dict(zip(\n        list(objects[0][0].params._getkeys()),\n        list(objects[0][0].params._getvalues()),\n    )) '